$$\require{mhchem}$$

# Adiabatic Flame Temperature: Complete Combustion with Cantera

The adiabatic flame temperature is the temperature achieved when a given fuel-oxidizer combination is combusted in a perfectly insulated reactor. If no energy is lost to the surroundings via heat transfer and the reaction occurs at constant pressure, conservation of energy dictates that the total enthalpy of the reactants is equal to the total enthalpy of the products. Defining _state 1_ as the reactants and _state 2_ as the products, we can write:

$$h_{\text{mix}, 2} = h_{\text{mix}, 1}$$

The enthalpy of the mixture has two components:

1. the enthalpy of formation: $\Delta^{\circ}_{\text{f}}h$ 
2. the sensible enthalpy: $h_i$

To make this more concrete, let's pick a specific reaction: stoichiometric combustion of methane, $\ce{CH4}$ with air:

$$\ce{CH4 + 2 O2 + $7.52$N2 <=> 2 H2O + CO2 + $7.52$N2}$$

From the reaction stoichiometry (note that the net stoichiometric coefficient for $\ce{N2}$ is zero), the heat of combustion released by this reaction is:

$$\begin{aligned}q_{\text{rxn}} &= -\sum_k \nu_k \Delta^{\circ}_{\text{f}}h_k\\
&= \Delta^{\circ}_{\text{f}}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} - \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O}\end{aligned}$$

where $\nu_k$ is the stoichiometric coefficient of species $k$, which is positive for products and negative for reactants. By definition, the heat of reaction is released at the reference temperature.

However, since the reaction must occur at constant enthalpy, the heat released must be stored in the products. In essence, the heat is used to heat the products from the initial, reference, temperature to a final temperature:

$$\begin{aligned}q_{\text{heating}} &= \sum_k\nu_{\text{prod},k}\left[h_k\left(T_2\right) - h_k\left(T_1\right)\right]\\
&= h_{\ce{CO2}}\left(T_2\right) - h_{\ce{CO_2}}\left(T_1\right) + 2\left[h_{\ce{H2O}}\left(T_2\right) - h_{\ce{H2O}}\left(T_1\right)\right] + 7.52\left[h_{\ce{N2}}\left(T_2\right) - h_{\ce{N_2}}\left(T_1\right)\right]\end{aligned}$$

where, again, states 1 and 2 represent the final and initial states, respectively. $\nu_{\text{prod}}$ is the product stoichiometric coefficient. Note that this assumes complete combution - there are no reactant species left to heat.

For a purely adiabatic reaction, we will have $q_{\rm rxn} = q_{\rm heating}$, and hence:

$$\sum_k\nu_{\text{prod},k}\left[h_k\left(T_2\right) - h_k\left(T_1\right)\right] + \sum_k \nu_k \Delta^{\circ}_{\text{f}}h_k = 0$$

or:

$$h_{\ce{CO2}}\left(T_2\right) - h_{\ce{CO_2}}\left(T_1\right) + 2\left[h_{\ce{H2O}}\left(T_2\right) - h_{\ce{H2O}}\left(T_1\right)\right] + 7.52\left[h_{\ce{N2}}\left(T_2\right) - h_{\ce{N_2}}\left(T_1\right)\right] + \Delta^{\circ}_{\text{f}}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} - \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O} = 0$$

Note that only a few variables are dependent on the final temperature $T_2$.  We might be tempted to write the above equation in terms of those variables, but let's hold off for now. Our next step really depends on what method we use to solve the problem. In short, we need a means of calculating the enthalpy values, so that we may determine which value of $T_2$ satisfies our adiabatic condition.  

## Cantera

We will now find the enthalpy values necessary to solve this problem using Cantera. As it happens, we do not need to explicitly find enthalpy values. Instead, we will use Cantera to solve for the equilibrium state of the mixture of reactants while constraining the number of species considered in the problem.

In general, performing calculations with Cantera involves three basic steps:

1. Identify or create a YAML-based input file.
2. Create Cantera phases and set initial conditions.
3. Run the calculation.

Let's start by creating an appropriate input file.

### 1. Create a Cantera Input File

For equilibrium calculations, Cantera requires thermodynamic data for the species in the problem. The possible formats for these data are described on the [Cantera website](https://cantera.org/science/science-species.html#sec-thermo-models). Fortunately, it is not usually necessary to write entire Cantera input files by hand. They are often [converted](https://cantera.org/tutorials/ck2yaml-tutorial.html) from the common Chemkin format and they can reference other Cantera input files.

Cantera 2.5 introduces a new format for the input files referred to as _YAML format_. YAML stands for [YAML Ain't Markup Language](https://yaml.org/) and is a general purpose, declarative data file format. There are two main structures in a YAML file:

1. **Mappings** associate a _key_ with a _value_
2. **Arrays** collect multiple data into a single _value_

<div class="alert alert-info" role="alert">
    
**Note:** These two structures are similar to the Python dictionary and list data types, respectively.

</div>

More information about the YAML syntax is available in [Cantera's documentation](https://cantera.org/tutorials/yaml/yaml-format.html). In Cantera, valid input files _must_ have a mapping with the **key** `phases`. The **value** of this mapping is an array of phase definitions, which tell Cantera where to find thermodynamic, chemical kinetic, and transport information for each species and reaction in your system.

In this problem, we need thermodynamic data for $\ce{CH4}$, $\ce{O2}$, $\ce{N2}$, $\ce{CO2}$, and $\ce{H2O}$. The data for these species can all be found in the GRI-3.0 reaction mechanism. You are going to create a YAML input file that imports the thermodynamic data for these species from GRI-3.0. The advantage of this approach is that you can restrict the species to be exactly the five we are interested, rather than _all_ the species from GRI-3.0.

In [1]:
# Add python code here

This string defines one phase with the name `methane-air` and thermodynamics modeled as an ideal gas. We have chosen to load data for each species from mapping with the key `species` in the `gri30.yaml` file. This file is included with Cantera, but you could use any input file that you develop to load species data.

Species can also be defined in the same file as the phase definition. This is usually done when converting from other formats.

On the last line, the text in the `yaml_input` variable is written to a file called `methane.yaml`.

### 2. Create Cantera phases and set initial conditions

We now have an input file named `methane.yaml`. Following step 2 of the procedure, we have to create the phase in Python.

First, we have to tell Python where it can find Cantera by using an `import` statement. Then, we create a Cantera `Solution`.

In [2]:
# Add python code here

You now have Cantera `Solution` with the Python variable name `gas`, representing an ideal gas phase object.

We next want to set the initial conditions using the temperature, pressure, and mole fractions:

1. Temperature: 298.15 K
2. Stoichiometric mixture of methane and air: 1 mole of $\ce{CH4}$ to 2 moles of ($\ce{O2 + $3.76$ N2}$)
3. Pressure: 1 atmosphere

To do this, we will use the `TPX` property. This lets you set temperature, pressure, and mole fractions simultaneously. The syntax looks like this:

```python
gas.TPX = T_1, P_1, X_1
```

where `T_1`, `P_1`, and `X_1` are variables we create to specify the relevant values.  

Temperature and pressure are relatively straightforwad: we set the numerical values in units of K and Pa, respectively. For pressure, Cantera includes a property called `one_atm` that has the value of one standard atmosphere in Pa.

In [3]:
# Add python code here

For the mole fraction values, we have two options:

1. an array of mole fractions, in the exact order specified in the input file
2. a string that specifies the mole fractions of individual species, in any order

For either option, Cantera's default behavior is to normalize the values so that the sum of all mole fractions equals one, so that only the relative values matter.

We will choose the second option, where the species names and mole fractions are separated by a colon, and entries for each species are separated by commas. This way, we don't have to remember the order of the species in the input file:

In [4]:
# Add python code here

We can verify that we set the state as intended by printing out the state of the `gas` object:

In [5]:
# Add python code here

From this, we can see that the temperature and pressure are set appropriately. Cantera has automatically calculated the mixture properties using the thermodynamic data in the input file.

Now let's store the initial enthalpy and initial pressure for later verification.

In [6]:
# Add python code here

### 3. Run the calcualtion

Finally, the adiabatic flame temperature is found by equilibrating the initial mixture. Recall that the adiabatic flame temperature is defined such that enthalpy and pressure are held constant during the reaction. Therefore, you have to tell Cantera that these are the values which should be held constant in the equilbrium solver.

To equilibrate a `Solution`, you can use the `equilibrate()` method. This method takes one argument, a string of two characters, which should be the properties to be held constant.

In [7]:
# Add python code here

Now the thermodynamic state of the mixture is at the equlibrium condition. We can confirm that the enthalpy and pressure have been held constant:

In [8]:
# Add python code here

These values are very nearly zero, so the required condition is satisfied. The adiabatic flame temperature is the temperature associated with the equlibrated object:

In [9]:
# Add python code here

We may also be interested in the equilibrium mole fractions. We expect that the equilbrium composition should be made entirely of the reaction products, $\ce{CO2}$, $\ce{H2O}$, and $\ce{N2}$.

In [10]:
# Add python code here

Note that not _all_ of the methane and oxygen are consumed in this equilibrium calculation. However, values on the order of $10^{-7}$ or 0.1 ppm are negligible for most practical purposes.

As expected, the mole fraction of $\ce{N2}$ is nearly constant since it is not participating in any reactions.